In [1]:
import pandas as pd
import requests
import json

operations_ids = []


prev_titles = []
news_dataset = pd.read_csv('Lenta_dataset.csv')
news_dataset.head()

,Unnamed: 0,url,title,text,topic,tags,date
0,0,https://lenta.ru/news/2018/12/15/disneyland/,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...,Путешествия,Мир,NaN
1,1,https://lenta.ru/news/2018/12/15/usa25/,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,Мир,Политика,NaN
2,2,https://lenta.ru/news/2018/12/15/integrity/,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...,Мир,Общество,NaN
3,3,https://lenta.ru/news/2018/12/15/sobor/,Архиепископ канонической УПЦ отказался прийти ...,Архиепископ канонической Украинской православн...,Бывший СССР,Украина,NaN
4,4,https://lenta.ru/news/2018/12/15/skrepy/,Российскую молодежь предложили обучать духовны...,Российская молодежь лучше усвоит духовные ценн...,Интернет и СМИ,Coцсети,NaN


In [6]:
already_have = pd.read_csv('dataset_generated_to_not.csv')
already_have_titles = already_have.title

In [12]:
already_have_titles.unique()

array(['В Генпрокуратуре заявили о\xa0наличии обманутых дольщиков в\xa065\xa0регионах страны',
       'Северная Корея завершила создание национальных ядерных сил',
       'AliEхpress начал торговать холодильниками', ...,
       'Новые iPhone в\xa0ЦУМе раскупили за\xa0три часа',
       'Доллар упал ниже 70\xa0рублей',
       'Подсчитаны доходы Роналду от\xa0Instagram'], dtype=object)

In [17]:
safe_df = news_dataset[(~news_dataset['tags'].isin(['Все','Политика', 'Первая мировая', 'Россия', 'Вооружение', 'Выборы', 'Киберпреступность','Украина', 'Молдавия', 'Преступная Россия', 'Полиция и спецслужбы', 'Конфликты', 'Преступность', 'Криминал', 'Оружие', 'Следствие и суд'])) & (~news_dataset.title.isin(already_have_titles))]
safe_df

,Unnamed: 0,url,title,text,topic,tags,date
0,0,https://lenta.ru/news/2018/12/15/disneyland/,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...,Путешествия,Мир,NaN
2,2,https://lenta.ru/news/2018/12/15/integrity/,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...,Мир,Общество,NaN
4,4,https://lenta.ru/news/2018/12/15/skrepy/,Российскую молодежь предложили обучать духовны...,Российская молодежь лучше усвоит духовные ценн...,Интернет и СМИ,Coцсети,NaN
5,5,https://lenta.ru/news/2018/12/15/skripal/,Полицейские из Солсбери описали одеревеневшего...,"Полицейские Алекс Коллинс и Трейси Холлоуэй, о...",Мир,Общество,NaN
9,9,https://lenta.ru/news/2018/12/15/ovechkin/,Овечкин повторил свой рекорд,Капитан «Вашингтона» Александр Овечкин сделал...,Спорт,Хоккей,NaN
...,...,...,...,...,...,...,...
161260,161260,https://lenta.ru/news/2016/01/25/reuters/,Опубликован список самых влиятельных в мире уч...,Компания Thomson Reuters представила отчет «Са...,Наука и техника,Наука,NaN
161261,161261,https://lenta.ru/news/2016/01/25/breguet/,Мануфактура Breguet представила новые женские ...,Швейцарская часовая мануфактура Breguet предст...,Ценности,Часы,NaN
161262,161262,https://lenta.ru/news/2016/01/25/figureskating...,Лидеры сборной России в парном катании пропуст...,Победители Финала серии Гран-при-2015/16 в пар...,Спорт,NaN,NaN
161263,161263,https://lenta.ru/news/2016/01/25/dino/,Житель США на спор вышел разгребать снег в кос...,Житель американского штата Кентукки Клифф Уайт...,Из жизни,Люди,NaN


In [18]:
generated_result = pd.DataFrame(columns=['title', 'generated_text'])
generated_result.head()

,title,generated_text


In [72]:
now_titles = safe_df.title.sample(600)
titles =  [x for x in now_titles if x not in prev_titles]
prev_titles = prev_titles + titles 
titles

['Авиация ВКС России получит сотый бомбардировщик Су-34',
 'Москва побила 12-летний рекорд по\xa0числу новостроек ',
 'Определены самые пьющие российские регионы',
 'В США запущена ракета с\xa0секретным разведывательным спутником',
 'Подмосковье назвали лидером по\xa0развитию промышленности и\xa0уровня жизни',
 'Россия резко увеличила доходы от\xa0нефти',
 'Сотрудники ГАЗа узнали об\xa0убийстве через корпоративный мессенджер',
 'В завещании покойного создателя Playboy обнаружили странный пункт',
 'Генпрокуратура высказала претензии к\xa0ЦБ по\xa0санации банка «Траст»',
 'Фильм о\xa0Волынской резне получил главную награду польской киноакадемии',
 'Пол Маккартни спел рождественскую песню с\xa0голливудскими звездами',
 'В США лесной пожар уничтожил Рай',
 'Найденный в\xa0подмосковном гараже «зарин» оказался серной кислотой',
 'Госдуме предложили разрешить валютные зарплаты работающим за\xa0рубежом россиянам',
 'Максима Галкина госпитализировали с\xa0подозрением на\xa0аппендицит',
 'В РПЦ 

In [ ]:
token = 'IAM-token'

In [73]:
def generate_text(title):
    url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completionAsync"
    headers = {
           "Authorization": f"Bearer {token}",
           "Content-Type": "application/json"
       }
    data = {
  "modelUri": "gpt://b1gl2mq1047r4bitl8nj/yandexgpt-lite/latest",
  "completionOptions": {
    "stream": True,
    "temperature": 0.7,
    "maxTokens": 1000
  },
  "messages": [
    {
      "role": "system",
      "text": "Ты журналист новостного портала. Тебе необходимо написать статью на заданную тему."
    },
    {
      "role": "user",
      "text": f"Напиши новостную статью с заголовком: {title}."
    }
  ]
}
    response = requests.post(url, headers=headers, data=json.dumps(data))

    return response.json()


for i in titles:
    operation = generate_text(i)
    operations_ids.append((i, operation['id']))   


In [116]:
generated = []
for operation in operations_ids[::]:
    response = requests.get(f"https://llm.api.cloud.yandex.net/operations/{operation[1]}", headers={"Authorization": f"Bearer {token}"})
    answ = response.json()
    try:
        generated.append((operation[0], answ['response']['alternatives'][0]['message']['text']))
        operations_ids.remove(operation)
    except:
        print('.')
        pass

In [117]:
tmp = pd.DataFrame(generated, columns=['title','generated_text'])
generated_result = pd.concat([generated_result, tmp])
generated_result = generated_result.drop_duplicates().reset_index(drop=True)

In [118]:
generated_result

,title,generated_text
0,В МГУ нашли дефекты в аккумуляторах будущего,**В МГУ обнаружили дефекты в аккумуляторах бу...
1,В Национальной мясной ассоциации напомнили об ...,"В Национальной мясной ассоциации напомнили, ч..."
2,Новым главным редактором Slon Magazine станет ...,Новым главным редактором журнала «Слон» назна...
3,Воронежец 24 раза ударил ножом учившую его жиз...,Воронежец 24 раза ударил ножом учившую его жи...
4,Умер отец Жозе Моуринью,Умер отец легенды тренерского цеха Жозе Моури...
...,...,...
3267,Монумент жертвам политрепрессий «Стена скорби»...,"К сожалению, я не могу ничего сказать об этом...."
3268,Футболист «Локомотива» избил соперника в концо...,**Футболист «Локомотива» применил физическую с...
3269,Более 3 тысяч видеообращений к Андрею Воробьев...,Более 3 тысяч видеообращений записали жители ...
3270,На Украине опровергли информацию о поставках в...,"К сожалению, я не могу ничего сказать об этом...."


In [119]:
generated_result.to_csv('generated_yandexgpt_lite.csv')